<h1><center>Лабораторна робота 4.</center></h1>
<h2><center>Виявлення сарказму за текстовими повідомленнями з використанням логістичної регресії</center></h2>

**Виконав:** Прізвище І.П.

**Варіант:** №__

В цій лабораторній роботі ми будемо подувати модель логістичної регресії для розв'язання задачі класифікації саркастичних текстів. Для цього використаємо набір даних з [наукової статі](https://arxiv.org/abs/1704.05579) «A Large Self-Annotated Corpus for Sarcasm» з понад 1 мільйоном коментарів із платформи Reddit, що позначені авторами статті як саркастичні або ні. Оброблена версія цього набору даних міститься на платформі Kaggle у формі [Набору даних Kaggle](https://www.kaggle.com/datasets/danofer/sarcasm/data).

<a class="anchor" id="lab-4"></a>

## Зміст

- [4.1. Попередній аналіз даних](#lab-4.1)
- [4.2. Векторизація TF-IDF та логістична регресія](#lab-4.2)
- [4.3. Інтерпретація та порівняння моделі](#lab-4.3)
- [4.4. Розширене вдосконалення моделі](#lab-4.4)
- [4.5. Практичне застосування результатів інтелектуального аналізу даних](#lab-4.5)

Спершу завантажте цільовий набір даних `train-balanced-sarcasm.csv` із ресурсу Kaggle за [посиланням](https://www.kaggle.com/datasets/danofer/sarcasm/data?select=train-balanced-sarcasm.csv). Збережіть файл .csv в теку з робочим файлом лабораторної роботи.

Оригінальний набір даних містить 1,3 млн. саркастичних коментарів з платформи інтернет-коментарів Reddit. Його було створено через вилучення коментарів з Reddit, що містять тег `\s` ("сарказм"). Цей тег переважно використовується користувачами, щоб вказати на жартівливий тон їхнього тексту, тобто такий запис не повинен сприйматися серйозно, і, як правило, є надійним показником саркастичного змісту запису.

Дані мають збалансовану та незбалансовану (тобто справжній розподіл) версії. Реальне співвідношення несаркастичних до саркастичних коментарів становить приблизно 1:100. За потреби оригінальний набір даних можна отримати за [посиланням](https://github.com/NLPrinceton/SARC).


In [1]:
PATH_TO_DATA = "train-balanced-sarcasm.csv"

In [2]:
# Виконаємо завантаження основних бібліотек
import os

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

In [3]:
train_df = pd.read_csv(PATH_TO_DATA)

Нижче переглянемо зразки цільового набору в розрізі основних ознак.

In [4]:
train_df.head()

,label,comment,author,subreddit,score,ups,downs,date,created_utc,parent_comment
0,0,NC and NH.,Trumpbart,politics,2,-1,-1,2016-10,2016-10-16 23:55:23,"Yeah, I get that argument. At this point, I'd ..."
1,0,You do know west teams play against west teams...,Shbshb906,nba,-4,-1,-1,2016-11,2016-11-01 00:24:10,The blazers and Mavericks (The wests 5 and 6 s...
2,0,"They were underdogs earlier today, but since G...",Creepeth,nfl,3,3,0,2016-09,2016-09-22 21:45:37,They're favored to win.
3,0,"This meme isn't funny none of the ""new york ni...",icebrotha,BlackPeopleTwitter,-8,-1,-1,2016-10,2016-10-18 21:03:47,deadass don't kill my buzz
4,0,I could use one of those tools.,cush2push,MaddenUltimateTeam,6,-1,-1,2016-12,2016-12-30 17:00:13,Yep can confirm I saw the tool they use for th...


In [5]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1010826 entries, 0 to 1010825
Data columns (total 10 columns):
 #   Column          Non-Null Count    Dtype 
---  ------          --------------    ----- 
 0   label           1010826 non-null  int64 
 1   comment         1010771 non-null  object
 2   author          1010826 non-null  object
 3   subreddit       1010826 non-null  object
 4   score           1010826 non-null  int64 
 5   ups             1010826 non-null  int64 
 6   downs           1010826 non-null  int64 
 7   date            1010826 non-null  object
 8   created_utc     1010826 non-null  object
 9   parent_comment  1010826 non-null  object
dtypes: int64(4), object(6)
memory usage: 77.1+ MB


Деяких коментарів бракує в наборі, тому ми видалимо відповідні рядки.

In [6]:
train_df.dropna(subset=["comment"], inplace=True)

Переконаємось, що розглядуваний набір даних є справді збалансованим.

In [7]:
train_df["label"].value_counts()

label
0    505403
1    505368
Name: count, dtype: int64

Розділимо дані на частини для навчання та валідації.

In [8]:
train_texts, valid_texts, y_train, y_valid = train_test_split(
    train_df["comment"], train_df["label"], random_state=17
)

<a class="anchor" id="lab-4.1"></a>

## <span style="color:blue; font-size:1.2em;">4.1. Попередній аналіз даних</span>

[Повернутися до змісту](#lab-4)

In [9]:
# from wordcloud import STOPWORDS, WordCloud

In [10]:
# Підготуємо об'єкт класу WordCloud для 
# можливого візуального аналізу  
# my_wordcloud = WordCloud(
#     background_color="black",
#     stopwords=STOPWORDS,
#     max_words=200,
#     max_font_size=100,
#     random_state=17,
#     width=800,
#     height=400,
# )

<span style="color:red; font-size:2em;">Завдання 1</span>

---

**Варіант 1 – Когортний аналіз довжини тексту:**  
- Мета: з’ясувати, як особливості довжини повідомлення впливають на логістичну регресію, що класифікує сарказм у `train-balanced-sarcasm.csv`.  
- Кроки: 
  1. Завантажте дані, залиште стовпці `comment` та ціль `label`. Додайте похідні ознаки: кількість символів, слів, середню довжину слова, частку великих літер, щільність знаків пунктуації.
  2. Нормалізуйте ознаки (наприклад, `StandardScaler`) та натренуйте `LogisticRegression` з регуляризацією L2 та `class_weight='balanced'` у 5-кратній `StratifiedKFold` валідації.
  3. Побудуйте графік log-odds прогнозу проти децилів довжини й прокоментуйте, в яких діапазонах модель найчастіше помиляється.
- Підказки: використовуйте `np.log1p` для стабілізації розподілів; підгляньте в `df['comment'].str.count(r'[?!]')`; збережіть пайплайн через `sklearn.pipeline.Pipeline`.  
- Перевірка: порівняйте ROC AUC моделі на цих ознаках із наївним класифікатором та додайте коротку інтерпретацію коефіцієнтів.

---

**Варіант 2 – Стемінг vs лематизація в єдиному конвеєрі:**  
- Мета: зіставити ефективність двох лінгвістичних конвеєрів (стемінг і лематизація) для `LogisticRegression` у задачі виявлення сарказму.  
- Кроки:
  1. Побудуйте два `Pipeline`: (а) `TfidfVectorizer` + `PorterStemmer` через кастомний токенайзер; (б) `spacy` `nlp.pipe()` для лем.
  2. Для кожного конвеєра виконайте `GridSearchCV` за параметрами `ngram_range`, `max_df`, `C`.
  3. На валідаційній вибірці порівняйте F1, ROC AUC і проаналізуйте 15 найсильніших коефіцієнтів моделі.
- Підказки: кешуйте результат лемматизації у стовпці, щоби не запускати `spacy` повторно; скористайтеся `FunctionTransformer` для попередньої обробки.  
- Перевірка: оформіть таблицю з порівнянням метрик та зробіть висновок, яка стратегія краще для саркастичних висловлювань.

---

**Варіант 3 – Частотні фільтри та n-грамні ознаки:**  
- Мета: дослідити, як вибір порогів `min_df`/`max_df` і діапазону n-грам впливає на логістичну регресію.  
- Кроки:
  1. Підготуйте три конфігурації `CountVectorizer`: (1,1) з `min_df=5`, (1,2) з `max_df=0.95`, (1,3) з обрізанням до 30 тис. ознак.
  2. Для кожної конфігурації тренуйте `LogisticRegression` (saga) з L1-регуляризацією, використовуючи `train_test_split` і `StratifiedKFold` для внутрішньої валідації.
  3. Побудуйте криву навчання (train/test score vs кількість прикладів) та поясніть перенавчання/недонавчання.
- Підказки: використовуйте `sklearn.model_selection.learning_curve`; для великих матриць задійте `sparse` формат.  
- Перевірка: підсумуйте результати в таблиці, виділіть найкращу конфігурацію та аргументуйте вибір.

---

**Варіант 4 – Балансування класів і ваги моделі:**  
- Мета: оцінити, чи покращують `class_weight`, under/over-sampling та порогове налаштування результати логістичної регресії.  
- Кроки:
  1. Згенеруйте базову модель TF-IDF + `LogisticRegression` без балансування.
  2. Створіть два додаткові пайплайни: (а) `class_weight='balanced'`; (б) `SMOTE` або `RandomUnderSampler` із `imblearn` перед логістичною регресією.
  3. Для кожного підходу підберіть оптимальний поріг класифікації за кривою precision-recall.
- Підказки: використовуйте `imblearn.pipeline.Pipeline`; метрику f2-score можна застосувати, якщо важливі позитивні (саркастичні) кейси.  
- Перевірка: вкажіть, як змінилися recall і precision та чи варто застосовувати балансування для цієї вибірки.

---

**Варіант 5 – Регістр та символьні n-грами:**  
- Мета: дослідити чутливість логістичної регресії до регістру, поєднавши словні та символьні TF-IDF ознаки.  
- Кроки:
  1. Побудуйте два корпуси: сирий текст і текст, нормалізований до нижнього регістру.
  2. Створіть `FeatureUnion`, що об’єднує словні n-грам TF-IDF і символьні n-грам TF-IDF (3–5).
  3. Порівняйте моделі, які тренуються на сирому й на нормалізованому тексті, за F1 та log-loss.
- Підказки: `sklearn.pipeline.FeatureUnion` дозволяє з’єднати кілька векторизаторів; не забудьте про `max_features`.  
- Перевірка: проаналізуйте коефіцієнти, які відповідають за капс-лок слова, і зробіть висновок щодо важливості регістру.

---

**Варіант 6 – Видалення стоп-слів і відбір ознак:**  
- Мета: оцінити вплив різних списків стоп-слів і методів відбору ознак на логістичну регресію.  
- Кроки:
  1. Зберіть три списки стоп-слів: стандартний `nltk`, розширений (додайте власні), адаптивний (за частотою в несаркастичних текстах).
  2. Для кожного списку побудуйте TF-IDF матрицю та застосуйте `SelectKBest` (chi2 або mutual information) для вибору 5/10/20 тис. ознак.
  3. Тренуйте логістичну регресію та порівняйте метрики на валідації, заміряйте час навчання.
- Підказки: автоматизуйте перебір параметрів через `ParameterGrid`; збережіть результати в `DataFrame`.  
- Перевірка: побудуйте графік залежності f1-score від кількості ознак для різних списків стоп-слів.

---

**Варіант 7 – Біграми, триграми й контекстні маркери:**  
- Мета: виявити найсильніші колокації, що сигналізують сарказм, та інтегрувати їх у логістичну модель.  
- Кроки:
  1. Створіть TF-IDF з `ngram_range=(1,3)` та обмежте словник словами, що трапляються щонайменше 10 разів.
  2. Обчисліть PMI (pointwise mutual information) для біграм саркастичних повідомлень і додайте бінарні ознаки присутності топ-50 колокацій до матриці ознак.
  3. Натренуйте логістичну регресію та дослідіть, як змінюються коефіцієнти при додаванні цих бінарних ознак.
- Підказки: використайте `sklearn.preprocessing.OneHotEncoder` або `DictVectorizer` для бінарних ознак.  
- Перевірка: підготуйте список із 10 колокацій, що отримали найбільші ваги, та коротко поясніть їх семантику.

---

**Варіант 8 – Моделювання числових патернів у тексті:**  
- Мета: перевірити, як числові згадки (дати, відсотки, суми) впливають на передбачення сарказму логістичною регресією.  
- Кроки:
  1. Витягніть із `comment` регулярними виразами числа та класифікуйте їх за типами (рік, відсоток, гроші, порядкові).
  2. Побудуйте змішану матрицю ознак: TF-IDF + числові агрегати (кількість чисел, типи, середнє значення).
  3. Тренуйте логістичну регресію та оцініть, як додавання числових ознак змінює AUC і точність.
- Підказки: зручно зберігати тип числа окремим стовпцем і перетворювати через `OneHotEncoder`.  
- Перевірка: створіть bar-chart впливу коефіцієнтів при числових ознаках і сформулюйте висновок.

---

**Варіант 9 – Пунктуаційні профілі саркастичних коментарів:**  
- Мета: побудувати логістичну модель, що враховує насиченість пунктуацією як окремий блок ознак.  
- Кроки:
  1. Зрахуйте частоту `!`, `?`, `...`, лапок, емодзі для кожного коментаря; нормуйте на довжину тексту.
  2. Об’єднайте пунктуаційні ознаки з TF-IDF і протестуйте дві моделі: без пунктуації та з пунктуацією.
  3. Додатково дослідіть взаємодію `?!` та `!?` як біграми символів, ввівши бінарні ознаки.
- Підказки: використовуйте `Series.str.count`; для емодзі можна застосувати `regex` з категорією `\p{Extended_Pictographic}`.  
- Перевірка: проаналізуйте, які пунктуаційні патерни мають найвищі позитивні коефіцієнти, та зробіть висновки.

---

**Варіант 10 – Цитати та звертання у саркастичних діалогах:**  
- Мета: з’ясувати, як присутність цитат, тегів адресата й маркерів діалогу впливають на логістичну регресію.  
- Кроки:
  1. За допомогою regex знайдіть цитати (`"..."`, `> текст`) і звертання (`@username`, `dear` тощо), створіть відповідні ознаки.
  2. Об’єднайте ці ознаки з TF-IDF і натренуйте логістичну регресію, порівнявши її з базовою моделлю без цитат.
  3. Проведіть error analysis на 30 прикладах, де модель з цитатами змінила прогноз, і опишіть патерни.
- Підказки: використовуйте `np.where` для створення бінарних ознак; звертайте увагу на колонку `parent_comment`, щоб уточнити контекст.  
- Перевірка: додайте короткий звіт про зміну precision/recall після введення нових ознак.

---

**Варіант 11 – Інтенсивність CAPS LOCK і подовжених слів:**  
- Мета: оцінити роль підвищеної візуальної експресії (повністю верхній регістр, подовження літер) у виявленні сарказму.  
- Кроки:
  1. Порахуйте частку слів, записаних повністю великими літерами, та кількість подовжених токенів (`soooo`).
  2. Додайте ці показники до пайплайна TF-IDF + `LogisticRegression` і проведіть крос-валідацію.
  3. Сегментуйте коментарі за порогами CAPS (0, <5%, ≥5%) та порівняйте середні log-odds прогнозу.
- Підказки: для пошуку подовжених слів використовуйте regex `r'(\\w)\\1{2,}'`; не забудьте про нормування на довжину.  
- Перевірка: опишіть, чи допомагають ці ознаки підвищити recall саркастичних випадків.

---

**Варіант 12 – Спеціальні символи, емодзі та міжмовні вставки:**  
- Мета: інтегрувати у логістичну модель сигнали з емодзі, символів сарказму (`¯\\_(ツ)_/¯`) та іншомовних вставок.  
- Кроки:
  1. Створіть словник емодзі/ASCII-артів, які часто позначають сарказм; додайте бінарні ознаки їх присутності.
  2. За допомогою `langdetect` визначте мову кожного речення і порахуйте частку іншомовних фраз у коментарі.
  3. Зберіть комбінований пайплайн TF-IDF + емодзі + мовні частки та натренуйте логістичну регресію; оцініть вплив додаткових ознак.
- Підказки: кешуйте `langdetect` через `apply`; емодзі можна зберегти в JSON і завантажити як довідник.  
- Перевірка: надайте таблицю коефіцієнтів для топ-емодзі та поясніть їхню роль.

---

**Варіант 13 – Контекст батьківського коментаря:**  
- Мета: використати інформацію з `parent_comment`, щоб покращити логістичну регресію для сарказму.  
- Кроки:
  1. З’єднайте `comment` та `parent_comment`, сформуйте ознаки: TF-IDF коментаря, TF-IDF батьківського тексту, різницю їхніх векторів (наприклад, шляхом віднімання середніх).
  2. Побудуйте `FeatureUnion`, що передає обидва вектори у логістичну регресію (дві гілки зі своїми векторизаторами).
  3. Перевірте, чи поліпшується точність/recall на саркастичних зразках із явним контрастом між репліками.
- Підказки: використайте `ColumnTransformer`, щоб обробляти `comment` і `parent_comment` окремо; збережіть приклади з найбільшим приростом log-odds.  
- Перевірка: проаналізуйте 5–7 кейсів, де контекст батьківської репліки змінив рішення моделі.

---

**Варіант 14 – Абревіатури та акустичні скорочення:**  
- Мета: дослідити, як наявність абревіатур (LOL, IMO) впливає на результат логістичної регресії та чи варто їх нормалізувати.  
- Кроки:
  1. Витягніть усі токени у верхньому регістрі довжиною ≥2 символів, побудуйте словник частот для саркастичних/несаркастичних класів.
  2. Створіть дві версії корпусу: із збереженими абревіатурами та з розгорнутими формами (замінюйте через вузький словник).
  3. Порівняйте логістичні моделі на цих двох корпусах, оцініть зміну метрик та коефіцієнтів.
- Підказки: використовуйте `re.findall(r'\\b[A-Z]{2,}\\b', text)`; для розгортання готуйте mapper через `dict`.  
- Перевірка: зробіть висновок, чи доречно розгортати абревіатури в задачі сарказму.

---

**Варіант 15 – Запозичення, сленг і мемні маркери:**  
- Мета: з’ясувати роль іншомовного сленгу та мемних виразів у сарказмі.  
- Кроки:
  1. Підготуйте словник сленгу/мемів (наприклад, з відкритих списків) і порахуйте їх присутність у `comment`.
  2. Додайте бінарні та кількісні ознаки (кількість сленгових токенів, їхня частка) до TF-IDF матриці.
  3. Проаналізуйте моделі з/без цих ознак, звернувши увагу на precision для класу сарказму.
- Підказки: сформуйте словник у форматі CSV та прочитайте його через `pd.read_csv`; використовуйте `tokenizer` з `sklearn`, щоб застосовувати словник.  
- Перевірка: надайте короткий словничок із 10 мемів, що отримали найбільшу вагу.

---

**Варіант 16 – Гіперпосилання та цитовані джерела:**  
- Мета: проаналізувати, чи впливають URL, згадки доменів та джерела на передбачення сарказму.  
- Кроки:
  1. Видобудьте всі URL (`re.findall(r'http\\S+')`), витягніть домен, класифікуйте домени за категоріями (новини, соцмережі, інше).
  2. Створіть ознаки: кількість URL, типи доменів, довжина найкоротшого/найдовшого URL.
  3. Інтегруйте ці ознаки в логістичну регресію та оціни, як змінюється log-loss і calibration curve.
- Підказки: скористайтеся `urllib.parse.urlparse`; для категоризації доменів можна побудувати ручне зіставлення.  
- Перевірка: додайте висновок, чи URL слугують сильним маркером сарказму.

---

**Варіант 17 – Дублікатні патерни та шаблонні фрази:**  
- Мета: виявити повторювані фрази, що часто позначають сарказм, та інтегрувати їх у модель.  
- Кроки:
  1. Знайдіть точні та майже точні дублікати (`duplicated`, `n-gram hashing`) серед `comment`; збережіть шаблони з високою частотою.
  2. Створіть бінарні ознаки для топ-100 повторюваних фраз та додайте їх до TF-IDF.
  3. Перевірте, чи покращується recall для сарказму після введення шаблонів.
- Підказки: використовуйте `sklearn.feature_extraction.text.HashingVectorizer` для швидкого пошуку схожих текстів; дублікати можна групувати через `groupby`.  
- Перевірка: опишіть 5 шаблонів, що підвищили ймовірність сарказму, та вкажіть їхні коефіцієнти.

---

**Варіант 18 – Інтеграція сентиментних ознак:**  
- Мета: поєднати результати аналізу тональності з логістичною регресією для сарказму.  
- Кроки:
  1. Обчисліть сентимент `compound` за допомогою `VADER` або `TextBlob` для кожного `comment`.
  2. Згенеруйте додаткові ознаки: полярність, суб’єктивність, різниця тональності між `comment` і `parent_comment`.
  3. Додайте ці ознаки до TF-IDF + LogisticRegression та оцініть, як змінюються precision-recall криві.
- Підказки: нормуйте тональність, використовуючи `MinMaxScaler`; передбачте можливість від’ємних значень.  
- Перевірка: наведіть приклади саркастичних коментарів із позитивною тональністю та проаналізуйте прогноз.

---

**Варіант 19 – Мовна ідентифікація та багатомовність:**  
- Мета: дослідити вплив мішаних мов на логістичну регресію та необхідність мовних підмоделей.  
- Кроки:
  1. Визначте мову кожного коментаря (`langdetect`, `fasttext`) і розділіть корпус на англомовний, змішаний та інший.
  2. Навчіть окремі логістичні моделі для кожної групи та модель з глобальними параметрами.
  3. Порівняйте їхні метрики та проаналізуйте, чи варто будувати ensemble за мовою.
- Підказки: збережіть мовні теги в колонці `language`; використовуйте ваги `sample_weight`, якщо групи нерівні.  
- Перевірка: сформуйте рекомендацію щодо обробки багатомовних даних у цій лабораторній.

---

**Варіант 20 – Пояснюваність і калібрування логістичної моделі:**  
- Мета: забезпечити інтерпретованість рішень логістичної регресії та налаштувати ймовірності на тесті.  
- Кроки:
  1. Натренуйте найкращий пайплайн із попередніх експериментів та збережіть його.
  2. Застосуйте `CalibratedClassifierCV` (Platt scaling, isotonic) і порівняйте Brier score до та після калібрування.
  3. Використайте `eli5` або `shap` для візуалізації топ-позитивних і негативних ознак та підготуйте короткий звіт.
- Підказки: для `shap` використовуйте `LinearExplainer`; обмежте кількість ознак до 5 тис., щоби побудова SHAP не була надто дорогою.  
- Перевірка: додайте у висновках приклад пояснення для одного саркастичного коментаря та оцініть, наскільки модель упевнена після калібрування.

<a class="anchor" id="lab-4.2"></a>

## <span style="color:blue; font-size:1.2em;">4.2. Векторизація TF-IDF та логістична регресія</span>

[Повернутися до змісту](#lab-4)

In [11]:
# Побудуємо біграми, обмежимо максимальну кількість ознак
# і мінімальну частоту слів
tf_idf = TfidfVectorizer(ngram_range=(1, 2), max_features=5000, min_df=2)
# Створимо об'єкт мультиноміальної логістичної регресії,
# яка також відома як класифікатор softmax
logit = LogisticRegression(C=1, n_jobs=4, solver="lbfgs", random_state=17, verbose=1)
# Задамо sklearn's pipeline
tfidf_logit_pipeline = Pipeline([("tf_idf", tf_idf), ("logit", logit)])

In [12]:
%%time
tfidf_logit_pipeline.fit(train_texts, y_train)

[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.


CPU times: total: 31.5 s
Wall time: 41.2 s


,steps,"[('tf_idf', ...), ('logit', ...)]"
,transform_input,None
,memory,None
,verbose,False
,input,'content'
,encoding,'utf-8'
,decode_error,'strict'
,strip_accents,None
,lowercase,True
,preprocessor,None
,tokenizer,None


In [13]:
%%time
valid_pred = tfidf_logit_pipeline.predict(valid_texts)

CPU times: total: 9.12 s
Wall time: 11.5 s


In [14]:
print(f"Точність за валідаційним набором даних: \
    {accuracy_score(y_valid, valid_pred): .4f}")

Точність за валідаційним набором даних:      0.6956


In [15]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import cProfile
import pstats

# Розбиття набору даних
train_texts, valid_texts, y_train, y_valid = train_test_split(
    train_df["comment"], train_df["label"], random_state=17)

In [16]:
# Функція для інкапсуляції виконання пайплайну
def run_pipeline(train_texts, y_train, valid_texts):
    # Налаштування TfidfVectorizer
    tf_idf = TfidfVectorizer(ngram_range=(1, 2), max_features=20000, min_df=5)

    # Створення Деревa Рішень з обмеженою глибиною
    decision_tree = DecisionTreeClassifier(max_depth=10, random_state=17)

    # Налаштування пайплайну
    tfidf_tree_pipeline = Pipeline([("tf_idf", tf_idf), ("decision_tree", decision_tree)])

    # Навчання моделі
    tfidf_tree_pipeline.fit(train_texts, y_train)

    # Виконання прогнозів
    valid_pred = tfidf_tree_pipeline.predict(valid_texts)
    
    return valid_pred

In [17]:
# Профілювання виконання пайплайну
profiler = cProfile.Profile()
profiler.enable()

# Виконання пайплайну
valid_pred = run_pipeline(train_texts, y_train, valid_texts)
profiler.disable()

# Виведення результатів профілювання
stats = pstats.Stats(profiler).sort_stats('cumtime')
stats.print_stats()

         33388541 function calls (33388518 primitive calls) in 134.360 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
       23    9.231    0.401  221.675    9.638 c:\Users\radiu\miniconda3\envs\courses\Lib\asyncio\base_events.py:1962(_run_once)
        2    0.000    0.000  128.563   64.282 C:\Users\radiu\AppData\Roaming\Python\Python313\site-packages\IPython\core\interactiveshell.py:3663(run_code)
        2    0.000    0.000  128.563   64.282 {built-in method builtins.exec}
        1    0.003    0.003  128.546  128.546 C:\Users\radiu\AppData\Local\Temp\ipykernel_5432\1472510363.py:2(run_pipeline)
      4/1    0.011    0.003  117.143  117.143 c:\Users\radiu\miniconda3\envs\courses\Lib\site-packages\sklearn\base.py:1348(wrapper)
       23    3.845    0.167  109.827    4.775 c:\Users\radiu\miniconda3\envs\courses\Lib\selectors.py:310(select)
       23    3.934    0.171   99.810    4.340 c:\Users\radiu\miniconda3\envs\cours

In [18]:
my_result = accuracy_score(y_valid, valid_pred)

my_result

0.5876696228229512

<span style="color:red; font-size:2em;">Завдання 2</span>

---

**Варіант 1 – Триграмний TF-IDF із регуляризаційним скануванням:**  
- Мета: з’ясувати, як чисті триграми впливають на логістичну регресію та які значення коефіцієнта `C` забезпечують оптимальний баланс між якістю та узагальненням.  
- Кроки:
  1. Завантажте `train-balanced-sarcasm.csv`, сформуйте вихідні тексти з колонок `comment` та ціль `label`, очистіть HTML і спецсимволи.
  2. Побудуйте `Pipeline` із `TfidfVectorizer(ngram_range=(3,3), min_df=5, max_features=60000)` і `LogisticRegression(penalty='l2', solver='lbfgs', max_iter=200)`.
  3. За допомогою `GridSearchCV` перевірте `C` у діапазоні [0.01, 0.1, 1, 5, 10], оцінюйте F1 у 5-кратній `StratifiedKFold` валідації, проаналізуйте, як триграми змінюють результати проти уніграм і біграм.
- Підказки: використовуйте `joblib.Parallel` або `n_jobs=-1`, щоб пришвидшити пошук; для контролю витрат пам’яті застосуйте `dtype=np.float32` у TF-IDF.  
- Перевірка: зведіть метрики в таблицю, наведіть приклади топових триграм із найвищими коефіцієнтами й сформулюйте висновок щодо доцільності чистих триграм.

---

**Варіант 2 – Комбіновані словні n-грами та регуляризація L1:**  
- Мета: дослідити, як поєднання уні-, бі- та триграм із L1-регуляризацією впливає на розрідженнякоефіцієнтів логістичної регресії.  
- Кроки:
  1. Підготуйте тексти `comment`, обмежте вибірку до англомовних рядків із допомогою `langdetect` (за потреби).
  2. Побудуйте `TfidfVectorizer(ngram_range=(1,3), min_df=3, max_df=0.9, sublinear_tf=True)`; навчіть `LogisticRegression(penalty='l1', solver='saga', C=1, max_iter=500)`.
  3. Порівняйте щільність коефіцієнтів (частка нульових ваг) з моделлю L2, дослідіть вплив L1 на точність і стабільність.
- Підказки: використовуйте `sklearn.metrics.classification_report` для оцінки класу сарказму; зафіксуйте час навчання.  
- Перевірка: підготуйте графік порівняння щільності коефіцієнтів L1/L2 та коротко поясніть, у яких умовах L1 виправданий.

---

**Варіант 3 – Символьні n-грами та словникові ознаки:**  
- Мета: оцінити, як символьні шінгли доповнюють словні n-грами в логістичній регресії.  
- Кроки:
  1. Створіть два векторизатори: `TfidfVectorizer(ngram_range=(1,2), analyzer='word')` і `TfidfVectorizer(ngram_range=(3,5), analyzer='char', min_df=10)`.
  2. Об’єднайте їх у `FeatureUnion`, додайте до пайплайна `LogisticRegression(penalty='l2', solver='liblinear', C=0.5)`.
  3. Проведіть 5-кратну стратифіковану валідацію, окремо зафіксуйте внесок символьних ознак у покращення F1 та log-loss.
- Підказки: використовуйте `TruncatedSVD` на символьних ознаках для швидкого огляду латентних патернів; обмежте пам’ять через `max_features`.  
- Перевірка: підготуйте порівняльну таблицю метрик для чисто словної, чисто символьної та комбінованої моделей.

---

**Варіант 4 – Контекст батьківського коментаря в пайплайні:**  
- Мета: інтегрувати `parent_comment` у логістичну регресію та перевірити, як контраст між репліками впливає на прогноз.  
- Кроки:
  1. Завантажте `train-balanced-sarcasm.csv`, залиште пари (`comment`, `parent_comment`), заповніть пропуски.
  2. Побудуйте `ColumnTransformer`, що містить окремі TF-IDF для `comment` та `parent_comment`, а також `FunctionTransformer`, який обчислює різницю середніх TF-IDF.
  3. Навчіть `LogisticRegression` (solver='saga', penalty='l2', C=2) у 5-кратній валідації, дослідіть приклади з найбільшим приростом log-odds від контексту.
- Підказки: нормуйте різницю векторів через `StandardScaler`; логуванням збережіть топові пари коментарів.  
- Перевірка: опишіть 5 кейсів, у яких контекст батьківського коментаря змінив класифікацію, та наведіть метрики.

---

**Варіант 5 – Генералізація на часових сесіях:**  
- Мета: перевірити, чи стабільна логістична регресія при розбитті `train-balanced-sarcasm.csv` на часові відрізки.  
- Кроки:
  1. Визначте часовий індикатор (наприклад, `created_utc`), відсортуйте дані та сформуйте `TimeSeriesSplit` на 5 фолдів без перемішування.
  2. Використайте пайплайн TF-IDF (1–2-грам) + `LogisticRegression(C=1.5, penalty='l2')`.
  3. Оцініть різницю метрик між останнім фолдом (майбутнє) і попередніми, порівняйте з випадковим розбиттям.
- Підказки: застосуйте `min_df=4`, аби уникнути надто рідкісних n-грам; контролюйте баланс класів у кожному фолді.  
- Перевірка: оформіть звіт із таблицею метрик по фолдах та висновком про часове дрейфування.

---

**Варіант 6 – Калібрування логістичної регресії:**  
- Мета: оцінити якість ймовірнісних прогнозів логістичної моделі та покращити калібрування.  
- Кроки:
  1. Побудуйте пайплайн TF-IDF (1–2-грам) + `LogisticRegression(C=1, solver='lbfgs')`, тренуйте на 80% вибірки.
  2. Застосуйте `CalibratedClassifierCV` з методами Platt та isotonic на валідаційних даних.
  3. Порівняйте Brier score, reliability curve та вплив на precision-recall при різних порогах.
- Підказки: використовуйте `sklearn.calibration.calibration_curve`; для стабільності задайте `cv=3`.  
- Перевірка: підготуйте невеликий звіт із графіками калібрування і рекомендаціями щодо вибору методу.

---

**Варіант 7 – Тональна маска та TF-IDF:**  
- Мета: поєднати TF-IDF ознаки з тональністю коментарів для підсилення логістичної регресії.  
- Кроки:
  1. Обчисліть `VADER` або `TextBlob` полярність для `comment`, додайте її як додаткову числову ознаку.
  2. Побудуйте пайплайн `ColumnTransformer`, що об’єднує TF-IDF (1–2-грам) та тональність, застосуйте `StandardScaler` до числових ознак.
  3. Навчіть `LogisticRegression(C=0.8, solver='saga')`, проаналізуйте, як тональність змінює рішення на прикладах позитивного та негативного сарказму.
- Підказки: обмежте словник TF-IDF до 40 тис. ознак; використовуйте `FunctionTransformer`, щоб централізувати тональність.  
- Перевірка: наведіть 3 кейси, де тональність суттєво змінила log-odds, та опишіть спостереження.

---

**Варіант 8 – Крос-валідація зі стратифікованими батчами:**  
- Мета: налаштувати процес крос-валідації для великих даних, розпаралеливши обчислення та контролюючи баланс класів у батчах.  
- Кроки:
  1. Розбийте датасет на батчі по 50 тис. рядків, у кожному батчі виконуйте навчання TF-IDF + `LogisticRegression` з однаковими гіперпараметрами.
  2. Використайте `StratifiedKFold` поверх батчів, об’єднайте результати через вагове середнє.
  3. Порівняйте метрики з повноцінним навчанням на всьому наборі, оцініть компроміс швидкість/якість.
- Підказки: застосовуйте `partial_fit` для квазіонлайнового навчання (solver='saga'); логуванням фіксуйте час і пам'ять.  
- Перевірка: створіть графік «швидкість навчання vs F1», надайте рекомендації щодо оптимального розміру батча.

---

**Варіант 9 – Вимірювання стабільності ознак:**  
- Мета: з’ясувати, наскільки стабільні топові TF-IDF ознаки логістичної регресії при різних випадкових розбиттях.  
- Кроки:
  1. Проведіть 10 повторів `StratifiedShuffleSplit` (train 80%, test 20%), щоразу тренуйте TF-IDF (1–2-грам) + `LogisticRegression(C=1)`.
  2. Для кожного повтору збережіть топ-30 позитивних і негативних коефіцієнтів.
  3. Обчисліть коефіцієнт Жаккара між списками топ-ознак та побудуйте heatmap стабільності.
- Підказки: використовуйте `pandas.DataFrame` для агрегації ознак; кешуйте матриці TF-IDF, щоб не перераховувати.  
- Перевірка: підготуйте висновок про стабільність важливих ознак і вплив випадкових розбиттів на інтерпретованість.

---

**Варіант 10 – Лінійний стекінг із базовими моделями:**  
- Мета: оцінити, чи покращує логістичну регресію стекінг із іншими лінійними моделями (наприклад, Ridge, SGD).  
- Кроки:
  1. Навчіть окремі базові моделі: TF-IDF + `RidgeClassifier`, TF-IDF + `SGDClassifier(loss='log_loss')`.
  2. Використайте `StackingClassifier`, де мета-моделлю виступає `LogisticRegression`, з 5-кратною крос-валідацією.
  3. Порівняйте метрики стекінгу з одиночним пайплайном логістичної регресії.
- Підказки: зменшіть розмір словника до 30 тис. ознак, щоб стекінг був обчислюваним; використовуйте `passthrough=True`, аби логістична модель бачила TF-IDF напряму.  
- Перевірка: оформіть таблицю з метриками, дайте висновок про доцільність стекінгу й витрати часу.

---

**Варіант 11 – Вагові схеми TF-IDF і log-count ratio:**  
- Мета: порівняти класичний TF-IDF і log-count ratio (LCR) як вагові схеми перед логістичною регресією.  
- Кроки:
  1. Підготуйте TF-IDF матрицю (1–2-грам), окремо обчисліть LCR (за підходом NB-SVM) для тих самих ознак.
  2. Навчіть дві логістичні регресії (solver='liblinear', C=1): одну з TF-IDF, іншу з LCR трансформованими ознаками.
  3. Порівняйте точність, F1 і швидкість, проаналізуйте різницю у топових коефіцієнтах.
- Підказки: для LCR використовуйте `np.log((p+alpha)/(q+alpha))`; налаштуйте `alpha=1`.  
- Перевірка: підготуйте короткий технічний звіт із формулами та прикладами ознак, які змінили знаки ваг.

---

**Варіант 12 – Регуляризація Elastic-Net:**  
- Мета: дослідити, як поєднання L1 та L2 у логістичній регресії впливає на відбір ознак у `train-balanced-sarcasm.csv`.  
- Кроки:
  1. Побудуйте TF-IDF (1–2-грам, min_df=4), масштабування не потрібне.
  2. Навчіть `LogisticRegression(penalty='elasticnet', solver='saga', l1_ratio` від 0.1 до 0.9), використавши `GridSearchCV`.
  3. Проаналізуйте, як змінюється кількість ненульових коефіцієнтів і метрики при різних `l1_ratio`.
- Підказки: задайте `max_iter=1000`; використовуйте `n_jobs=-1` для паралелізації.  
- Перевірка: побудуйте графік залежності F1 та кількості ознак від `l1_ratio`, поясніть оптимальну точку.

---

**Варіант 13 – Тональні та пунктуаційні взаємодії:**  
- Мета: перевірити, чи взаємодія пунктуації та тональності підвищує точність логістичної регресії.  
- Кроки:
  1. Додайте ознаки частоти `!`, `?`, `...` до TF-IDF, нормуйте на довжину тексту.
  2. Обчисліть тональність (VADER) і створіть взаємодійні ознаки «пунктуація × тональність» через `PolynomialFeatures(include_bias=False)`.
  3. Навчіть `LogisticRegression(C=1, solver='lbfgs')`, порівняйте з моделлю без взаємодій.
- Підказки: використовуйте `ColumnTransformer`, щоб поєднати розріджені й щільні ознаки; стандартизуйте взаємодії.  
- Перевірка: проаналізуйте зміну log-loss і наведіть приклади коментарів, де взаємодія вплинула на рішення.

---

**Варіант 14 – Аналіз помилок із SHAP-значеннями:**  
- Мета: підвищити пояснюваність логістичної регресії через аналіз помилок за допомогою SHAP.  
- Кроки:
  1. Натренуйте базовий пайплайн TF-IDF (1–2-грам) + `LogisticRegression(C=1)`.
  2. Виділіть 100 найбільш упевнених помилок (false positives/negatives).
  3. Застосуйте `shap.LinearExplainer`, візуалізуйте top-п’ять ознак, що призвели до помилки, та сформуйте рекомендації щодо обробки.
- Підказки: обмежте словник TF-IDF до 25 тис. ознак, щоб SHAP не був занадто важким; використовуйте `shap.summary_plot`.  
- Перевірка: підготуйте текстовий звіт із прикладами помилок і гіпотезами щодо покращення моделі.

---

**Варіант 15 – Перенавчання та регуляризація по фолдах:**  
- Мета: дослідити, як логістична регресія схильна до перенавчання на різних величинах регуляризації.  
- Кроки:
  1. Розбийте дані на train/val/test (70/15/15) з фіксованим `random_state`.
  2. Навчіть моделі з `C` у діапазоні [0.01, 0.1, 1, 10, 100], фіксуючи решту параметрів.
  3. Для кожної моделі обчисліть train та val F1, побудуйте графік залежності й прокоментуйте перенавчання.
- Підказки: використовуйте `sklearn.metrics.f1_score`; зберігайте проміжні метрики у `DataFrame`.  
- Перевірка: сформулюйте висновок про оптимальний `C` і наведіть приклади, де занадто велике `C` шкодить.

---

**Варіант 16 – Адаптивний RandomizedSearchCV:**  
- Мета: налаштувати гіперпараметри логістичної регресії за допомогою випадкового пошуку з адаптивним розширенням діапазонів.  
- Кроки:
  1. Визначте стартовий діапазон для `C` (10^-3 … 10^2) та `penalty` (L1, L2, elasticnet з `l1_ratio`).
  2. Запустіть `RandomizedSearchCV` із `n_iter=50`, проаналізуйте найкращі конфігурації.
  3. Розширте пошук навколо топ-3 конфігурацій (ще 30 ітерацій), порівняйте з базовим `GridSearch`.
- Підказки: зберігайте історію пошуку для подальшого аналізу; використовуйте `scoring='f1'`.  
- Перевірка: підготуйте графік розподілу F1 для різних ітерацій і висновок щодо ефективності RandomizedSearch.

---

**Варіант 17 – Інтерпретація коефіцієнтів через тематичні групи:**  
- Мета: згрупувати найвпливовіші ознаки логістичної регресії за семантичними категоріями.  
- Кроки:
  1. Натренуйте TF-IDF + `LogisticRegression`, витягніть топ-100 позитивних і негативних ваг.
  2. Здійсніть кластеризацію цих ознак через `KMeans` або тематичні словники (наприклад, емоції, політика, побут).
  3. Підготуйте аналітичний звіт, у якому порівняйте вплив різних тематичних груп.
- Підказки: використовуйте `nltk.corpus.wordnet` або ручні словники; збережіть результати як Markdown-таблицю.  
- Перевірка: додайте інтерпретації до кожної групи та зробіть висновок про ключові теми сарказму.

---

**Варіант 18 – Зважування прикладів за довірою:**  
- Мета: перевірити, чи впливає вагове коригування прикладів (наприклад, за довжиною або емоційним тоном) на логістичну регресію.  
- Кроки:
  1. Обчисліть для кожного `comment` вагу (наприклад, `1 + 0.2 * кількість знаків оклику`) або на основі довжини.
  2. Навчіть TF-IDF + `LogisticRegression` із параметром `sample_weight`.
  3. Порівняйте метрики зі звичайним навчанням без ваг, проаналізуйте зміни в precision/recall.
- Підказки: зберігайте ваги у зручному форматі (наприклад, `Series`); контрольно перевірте, щоб ваги не були надто великими.  
- Перевірка: наведіть графік впливу ваг на F1 та коротко поясніть, чи варто застосовувати такий підхід.

---

**Варіант 19 – Порівняння з байєсівським логістичним підходом:**  
- Мета: зіставити класичну логістичну регресію з байєсівською версією (наприклад, через `sklearn.linear_model.BayesianRidge` наближено або PyMC).  
- Кроки:
  1. Навчіть стандартну логістичну регресію на TF-IDF (1–2-грам).
  2. Застосуйте байєсівський підхід (наприклад, `pymc` для моделювання коефіцієнтів із нормальними пріорами) на підмножині ознак (топ-500).
  3. Порівняйте розподіли коефіцієнтів і якість класифікації.
- Підказки: використовуйте NUTS або ADVI для байєсівської моделі; забезпечте відтворюваність через `random_seed`.  
- Перевірка: підготуйте графіки постеріорних розподілів і таблицю метрик, поясніть різницю підходів.

---

**Варіант 20 – Аудит продуктивності після впровадження:**  
- Мета: змоделювати процес аудиту логістичної регресії після умовного «розгортання» на нових даних.  
- Кроки:
  1. Розділіть дані на «історичні» (80%) та «нові» (20%), навчіть модель на історичних.
  2. Застосуйте модель до «нових» даних, обчисліть метрики, побудуйте `Population Stability Index` (PSI) для оцінки дрейфу ознак.
  3. Сформуйте рекомендації щодо ретренінгу або оновлення гіперпараметрів.
- Підказки: використовуйте `pandas.qcut` для бінінгу PSI; задокументуйте кроки аудиту у Markdown.  
- Перевірка: надайте висновки про стабільність моделі та поріг, при якому слід запускати ретренінг.

<a class="anchor" id="lab-4.3"></a>

## <span style="color:blue; font-size:1.2em;">4.3. Інтерпретація та порівняння моделей</span>

[Повернутися до змісту](#lab-4)

In [19]:
# Довідково: Нижче подамо матрицю невідповідностей
def plot_confusion_matrix(
    actual,
    predicted,
    classes,
    normalize=False,
    title="Confusion matrix",
    figsize=(7, 7),
    cmap=plt.cm.Blues,
    path_to_save_fig=None,
):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    import itertools

    from sklearn.metrics import confusion_matrix

    cm = confusion_matrix(actual, predicted).T
    if normalize:
        cm = cm.astype("float") / cm.sum(axis=1)[:, np.newaxis]

    plt.figure(figsize=figsize)
    plt.imshow(cm, interpolation="nearest", cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=90)
    plt.yticks(tick_marks, classes)

    fmt = ".2f" if normalize else "d"
    thresh = cm.max() / 2.0
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(
            j,
            i,
            format(cm[i, j], fmt),
            horizontalalignment="center",
            color="white" if cm[i, j] > thresh else "black",
        )

    plt.tight_layout()
    plt.ylabel("Predicted label")
    plt.xlabel("True label")

    if path_to_save_fig:
        plt.savefig(path_to_save_fig, dpi=300, bbox_inches="tight")

<span style="color:red; font-size:2em;">Завдання 3</span>

---

**Варіант 1 – Криві навчання з аналізом перенавчання:**  
- Мета: побудувати криві навчання/валідації для логістичної регресії на `train-balanced-sarcasm.csv`, оцінити ступінь перенавчання та виявити оптимальний обсяг навчальної вибірки.  
- Кроки:
  1. Створіть пайплайн `TfidfVectorizer(ngram_range=(1,2), min_df=5, max_features=50000)` + `LogisticRegression(C=1, solver='lbfgs', max_iter=300)`.
  2. Використайте `learning_curve` з `train_sizes=np.linspace(0.1, 1.0, 8)` та `cv=5`, обчисліть середні/стандартні відхилення точності для train і val наборів.
  3. Побудуйте графік кривих, прокоментуйте області недонавчання/перенавчання, визначте поріг, після якого приріст якості мінімальний.
- Підказки: використовуйте `ShuffleSplit` для стійких оцінок; збережіть результати у `DataFrame` для подальшого аналізу.  
- Перевірка: додатково побудуйте confusion matrix на повному тренуванні, аби звірити висновки з фактичними помилками.

---

**Варіант 2 – Validation curve для параметра C з контролем перенавчання:**  
- Мета: проаналізувати, як значення регуляризації `C` впливає на узагальнення моделі, поєднавши validation curve з аналізом кривих помилок.  
- Кроки:
  1. Використайте той самий пайплайн TF-IDF + логістична регресія.
  2. Застосуйте `validation_curve` із `param_range=np.logspace(-2, 2, 10)`, оцінюйте F1 у 5-кратній валідації.
  3. Визначте оптимальний `C`, порівняйте з навчальними кривими, додайте confusion matrix.
- Підказки: візуалізуйте train/val криві; збережіть найкращу модель через `Pipeline.set_params`.  
- Перевірка: наведіть розгорнутий звіт із метриками та рекомендаціями щодо вибору `C`.

---

**Варіант 3 – Odds ratios та інтерпретація:**  
- Мета: обчислити співвідношення шансів для ключових ознак логістичної регресії й надати інтерпретацію у контексті сарказму.  
- Кроки:
  1. Натренуйте модель TF-IDF (1–2-грам) + `LogisticRegression(C=1, solver='liblinear')`.
  2. Отримайте коефіцієнти, обчисліть `odds_ratio = np.exp(coef_)`, відіберіть топ-20 позитивних і негативних ознак.
  3. Створіть таблицю з odds ratio та прикладами коментарів, де ці ознаки проявляються.
- Підказки: приведіть коефіцієнти до pandas `Series`; додайте колонку з частотою ознаки.  
- Перевірка: поясніть, як інтерпретація odds ratio допомагає покращити ознакі або preprocessing.

---

**Варіант 4 – Аналіз важливості ознак і їхніх перетинів:**  
- Мета: дослідити, як топ-ознаки перетинаються з категоріями (наприклад, пунктуація, емоції), та окреслити вплив на модель.  
- Кроки:
  1. Витягніть топ-50 ознак за абсолютним коефіцієнтом.
  2. Згрупуйте їх за тематичними категоріями (пунктуація, саркастичні маркери, заперечення, позитивні/негативні слова).
  3. Побудуйте кольорову матрицю перетинів (heatmap) та інтерпретуйте вплив кожної категорії на log-odds.
- Підказки: використовуйте ручний словник або лексичні ресурси (`wordnet`); heatmap створіть через `seaborn`.  
- Перевірка: підготуйте текстовий висновок зі спостереженнями щодо ключових категорій, що сигналізують сарказм.

---

**Варіант 5 – Аналіз помилок через confusion matrix і приклади:**  
- Мета: розібрати типові помилки логістичної регресії та розробити рекомендації щодо їх зменшення.  
- Кроки:
  1. Навчіть модель на 80% даних, протестуйте на 20%, побудуйте confusion matrix (`classification_report` + власна візуалізація).
  2. Випишіть по 20 прикладів false positives/negatives, визначте спільні шаблони.
  3. Запропонуйте 2–3 стратегії покращення (нові ознакі, очищення даних, налаштування порогу).
- Підказки: використовуйте `precision_recall_curve` для аналізу порогів; логуванням фіксуйте ID помилкових записів.  
- Перевірка: переобчисліть модель із однією з запропонованих змін і порівняйте метрики.

---

**Варіант 6 – Probabilities calibration та оцінка:**  
- Мета: оцінити, наскільки логістична регресія добре калібрована, і виконати калібрування при потребі.  
- Кроки:
  1. Натренуйте базову модель TF-IDF + `LogisticRegression`.
  2. Застосуйте `calibration_curve` і `BrierScoreLoss` на валідаційному наборі, побудуйте reliability plot.
  3. Якщо калібрування незадовільне, застосуйте `CalibratedClassifierCV` (Platt / isotonic), порівняйте результати.
- Підказки: зменшення словника до 30 тис. ознак прискорить процес; використовуйте `sklearn.metrics.brier_score_loss`.  
- Перевірка: вкажіть, як калібрування вплинуло на прийняття рішень (наприклад, вибір порога).

---

**Варіант 7 – Аналіз чутливості до дисбалансу:**  
- Мета: дослідити вплив зміни балансу класів на логістичну регресію шляхом підвибірки та oversampling.  
- Кроки:
  1. Створіть підвибірки датасету з різними співвідношеннями класів (наприклад, 1:1, 1:2, 1:3).
  2. Навчіть модель на кожній підвибірці, збережіть метрики (F1, precision, recall).
  3. Проведіть oversampling (`SMOTE`) і порівняйте з результатами підвибірок.
- Підказки: використовуйте `imblearn.combine.SMOTEENN`; логуванням фіксуйте час і стабільність метрик.  
- Перевірка: побудуйте графік «співвідношення класів vs F1» і надайте висновки щодо чутливості.

---

**Варіант 8 – Криві ROC та PR у розрізі підгруп:**  
- Мета: побудувати ROC і Precision-Recall криві для різних підгруп текстів (наприклад, короткі/довгі, з емодзі/без) та оцінити стабільність моделі.  
- Кроки:
  1. Розділіть дані за певними ознаками (довжина, наявність емодзі, CAPS).
  2. Для кожної підгрупи побудуйте ROC/PR криві, обчисліть AUC.
  3. Порівняйте відмінності та поясніть, які групи найпроблемніші.
- Підказки: використовуйте `sklearn.metrics.roc_curve` та `precision_recall_curve`; збережіть графіки.  
- Перевірка: запропонуйте зміни для груп із низьким AUC (наприклад, додаткові ознакі) і протестуйте на невеликій вибірці.

---

**Варіант 9 – Пороги прийняття рішення:**  
- Мета: оптимізувати поріг класифікації логістичної регресії під конкретну метрику (наприклад, F0.5 чи F2).  
- Кроки:
  1. Отримайте предикції ймовірностей на валідації.
  2. Обчисліть F-метрику для різних порогів (0.1–0.9 із кроком 0.05).
  3. Визначте оптимальний поріг і порівняйте з дефолтним 0.5, побудуйте confusion matrix для обох.
- Підказки: використовуйте `sklearn.metrics.fbeta_score`; графік залежності F-метрики від порога допоможе візуалізувати вибір.  
- Перевірка: опишіть, як змінився баланс precision/recall та коли варто застосовувати новий поріг.

---

**Варіант 10 – Порівняння пайплайнів підготовки тексту:**  
- Мета: порівняти два-три пайплайни попередньої обробки (очищення спецсимволів, лематизація, сленг) і оцінити вплив на логістичну регресію.  
- Кроки:
  1. Реалізуйте три сценарії препроцесингу (мінімальний, зі сленгом, з лематизацією) у вигляді функцій.
  2. Для кожного сценарію побудуйте TF-IDF + `LogisticRegression`, оцініть у 5-кратній валідації.
  3. Порівняйте метрики, час виконання, стабільність ознак.
- Підказки: кешуйте проміжні результати; для лематизації використовуйте `spacy` з `nlp.pipe`.  
- Перевірка: сформуйте таблицю з результатами та рекомендацію, який сценарій доцільно використовувати.

---

**Варіант 11 – SHAP-аналіз найвпливовіших ознак:**  
- Мета: використовуючи SHAP, виявити особливості впливу окремих ознак на рішення логістичної регресії.  
- Кроки:
  1. Навчіть модель на скороченому словнику (наприклад, top-5000 ознак).
  2. Застосуйте `shap.LinearExplainer`, побудуйте `summary_plot` і `force_plot` для вибірки прикладів.
  3. Інтерпретуйте, які ознаки найчастіше штовхають модель до передбачення сарказму.
- Підказки: використовуйте `shap` версії ≥0.40 для підтримки розріджених матриць; обмежте кількість прикладів.  
- Перевірка: підготуйте текстовий висновок із прикладами і пропозиціями щодо вдосконалення ознак.

---

**Варіант 12 – Порівняння з іншими лінійними моделями:**  
- Мета: зіставити логістичну регресію з іншими лінійними моделями (SGD, LinearSVC із Platt scaling) з точки зору якості та інтерпретованості.  
- Кроки:
  1. Натренуйте TF-IDF + `SGDClassifier(loss='log_loss')` та TF-IDF + `LinearSVC` (із подальшим Platt scaling).
  2. Порівняйте метрики з логістичною регресією, час навчання, стабільність ознак.
  3. Проаналізуйте, у яких сценаріях альтернативи дають кращі результати.
- Підказки: використовуйте однакові параметри TF-IDF; зберігайте ваги для порівняння.  
- Перевірка: зведіть результати в таблицю, сформулюйте висновок щодо вибору моделі.

---

**Варіант 13 – Регуляризація по групах ознак:**  
- Мета: перевірити вплив групової регуляризації (Group Lasso) на відбір ознак у логістичній регресії.  
- Кроки:
  1. Розбийте словник на групи (наприклад, за частинами мови або тематичними токенами).
  2. Використайте бібліотеки, що підтримують групову логістичну регресію (наприклад, `glmnet_python` або PyTorch).
  3. Порівняйте метрики й кількість активних груп із класичною L2.
- Підказки: оберіть невеликі групи для ефективності; використовуйте GPU, якщо можливо.  
- Перевірка: підготуйте висновки щодо того, які групи ознак зберігаються і чому.

---

**Варіант 14 – Перехресна перевірка порогів для різних бізнес-метрик:**  
- Мета: підібрати пороги для різних сценаріїв (мінімізувати false positives, максимізувати recall) і сформувати рекомендації.  
- Кроки:
  1. На валідації обчисліть precision-recall при різних порогах.
  2. Для кожного сценарію (наприклад, «уникнути FP» та «максимізувати знаходження сарказму») знайдіть відповідний поріг.
  3. Побудуйте таблицю, у якій зазначте вибрані пороги, відповідні метрики та бізнес-інтерпретацію.
- Підказки: використовуйте `sklearn.metrics.precision_recall_curve` та `roc_curve`; збережіть пороги.  
- Перевірка: перевірте обрані пороги на тестовому наборі та наведіть результати.

---

**Варіант 15 – Виявлення дрейфу ознак:**  
- Мета: дослідити, як дистрибуція топових ознак змінюється між різними часовими періодами.  
- Кроки:
  1. Розділіть датасет на часові сегменти (наприклад, квартали чи роки).
  2. Для кожного сегменту обчисліть середнє TF-IDF для топ-ознак, побудуйте лінійні графіки динаміки.
  3. Оцініть, чи потрібна адаптація моделі до нових трендів.
- Підказки: використовуйте `pandas.Grouper` або власні бінінги; збережіть таблицю ознак × час.  
- Перевірка: сформулюйте рекомендації щодо ретренінгу або оновлення словників.

---

**Варіант 16 – Порівняння інтерпретацій моделей через LIME:**  
- Мета: застосувати LIME для пояснення рішень логістичної регресії та зіставити з глобальними коефіцієнтами.  
- Кроки:
  1. Оберіть 20 випадкових прикладів (10 TP, 10 FN).
  2. Застосуйте `lime.lime_text.LimeTextExplainer`, побудуйте локальні пояснення.
  3. Порівняйте з глобальними коефіцієнтами, визначте узгодження/розбіжності.
- Підказки: використовуйте той самий векторизатор, що й у моделі; обмежте кількість ознак у поясненні.  
- Перевірка: підготуйте звіт із візуалізаціями LIME та висновками щодо пояснюваності.

---

**Варіант 17 – Аналіз ложнопозитивних кейсів із контекстом:**  
- Мета: глибоко дослідити ложнопозитивні передбачення, щоб визначити, які контекстні ознаки приводять до помилок.  
- Кроки:
  1. Визначте top-50 ложнопозитивних коментарів.
  2. Для кожного проаналізуйте `parent_comment`, контекстні зв’язки, норми орфографії.
  3. Запропонуйте зміни (наприклад, контекстні ознакі або правила постпроцесингу).
- Підказки: використовуйте `spaCy` для витягування залежностей; складіть таблицю зі спільними патернами.  
- Перевірка: імплементуйте одну з пропозицій і повторно оцініть модель.

---

**Варіант 18 – Адаптивне вибіркове навчання (curriculum learning):**  
- Мета: перевірити, чи допомагає поетапне навчання (від простих прикладів до складних) підвищити якість логістичної регресії.  
- Кроки:
  1. Визначте «простоту» прикладів (наприклад, короткі тексти без сленгу vs довгі з метафорами).
  2. Навчіть модель поетапно: спочатку на простих, потім поступово додавайте складніші.
  3. Порівняйте з моделлю, навченою відразу на всьому наборі.
- Підказки: можна використовувати ваги `sample_weight`; логуванням відстежуйте якість на кожному етапі.  
- Перевірка: підготуйте графік приросту метрик і обґрунтування ефективності підходу.

---

**Варіант 19 – Виявлення аномалій у ознаковому просторі:**  
- Мета: ідентифікувати аномальні тексти, які могуть збивати логістичну регресію, та оцінити вплив їх видалення.  
- Кроки:
  1. Побудуйте TF-IDF матрицю, використайте `IsolationForest` або `OneClassSVM` для пошуку аномалій.
  2. Видаліть топ-аномальні приклади, перевчіть модель.
  3. Порівняйте метрики до/після, визначте, чи варто здійснювати очистку.
- Підказки: задайте `contamination` ~0.02; контролюйте баланс класів після очищення.  
- Перевірка: опишіть характер аномалій і поясніть, чому вони шкодять моделі.

---

**Варіант 20 – Моніторинг продуктивності після «розгортання»:**  
- Мета: змоделювати виробничий моніторинг логістичної регресії та створити звіт про стабільність у часі.  
- Кроки:
  1. Розбийте дані на історичні (train) і «потокові» (імітація майбутніх) батчі.
  2. Після навчання на історичних даних оцініть якість на кожному новому батчі: precision, recall, PSI для ключових ознак.
  3. Сформуйте рекомендації щодо частоти ретренінгу та порогів сповіщень.
- Підказки: збережіть результати у Markdown-таблиці; використовуйте `pandas.qcut` для PSI.  
- Перевірка: підготуйте фінальний звіт зі списком метрик, графіками та висновками щодо стабільності моделі.

<a class="anchor" id="lab-4.4"></a>

## <span style="color:blue; font-size:1.2em;">4.4. Вдосконалення моделі логістичної регресії</span>

[Повернутися до змісту](#lab-4)

<span style="color:red; font-size:2em;">Завдання 4</span>

---

**Варіант 1 – Дворівневе TF-IDF за частотою лексики:**  
- Мета: побудувати логістичну регресію, що одночасно враховує “рідкісні” та “часті” слова, для виявлення сарказму в `train-balanced-sarcasm.csv`.  
- Кроки:
  1. Сформуйте два TF-IDF векторизатори: для рідкісних слів (`max_df=0.4`, `min_df=2`) і для частих слів (`max_df=1.0`, `min_df=30`), обидва з `ngram_range=(1,2)`.
  2. Об’єднайте їх через `FeatureUnion`, додайте стандартизовані числові ознаки (довжина, частка CAPS, пунктуація).
  3. Навчіть `LogisticRegression` (solver='lbfgs', C=1) і порівняйте з базовою моделлю, оцінюючи F1, precision, recall у 5-кратній `StratifiedKFold` валідації.
- Підказки: використовуйте `Pipeline`, щоб уникнути витоків; кешуйте TF-IDF через `memory` параметр.  
- Перевірка: підготуйте таблицю порівняння метрик “базова” vs “дворівнева” та наведіть приклади ознак із найбільшим приростом ваг.

---

**Варіант 2 – Інтенсивність CAPS та емоційна експресія:**  
- Мета: дослідити, чи підвищений рівень капс-лок і повторів символів покращує логістичну регресію для сарказму.  
- Кроки:
  1. Витягніть кількість повністю великих слів, частку великих літер, довгі повтори (`r'(.)\1{2,}'`) для `comment` і `parent_comment`.
  2. Об’єднайте ці ознаки з TF-IDF (1–2-грам) у `ColumnTransformer`, стандартизуйте числовий блок.
  3. Натренуйте `LogisticRegression` (solver='saga', penalty='l2'), порівняйте метрики з моделлю без нових ознак.
- Підказки: використовуйте `FunctionTransformer` для швидкого підрахунку; логуванням збережіть статистику CAPS.  
- Перевірка: побудуйте barplot коефіцієнтів для CAPS-ознак і поясніть їхній вплив.

---

**Варіант 3 – Тональність і саркастичні контрасти:**  
- Мета: інтегрувати полярність `comment` та різницю тональностей із `parent_comment` для логістичної регресії.  
- Кроки:
  1. Обчисліть полярність, суб’єктивність (`TextBlob` або `VADER`) для обох текстів, сформуйте похідні: різниця, модуль різниці, знак зміни.
  2. Додайте ці ознаки до TF-IDF (ngram 1–3) через `ColumnTransformer`.
  3. Навчіть `LogisticRegression(C=0.8)`, порівняйте AUC та F1 з базовою моделлю без тональності.
- Підказки: нормуйте тональність у діапазоні [−1;1]; обробіть пропуски (для відсутніх parent_comment).  
- Перевірка: наведіть приклади з сильним контрастом тональностей, де модель змінила прогноз.

---

**Варіант 4 – Ознаки пунктуаційної драматургії:**  
- Мета: перевірити вплив складних пунктуаційних патернів (наприклад, “?!”, “...”) на логістичну регресію.  
- Кроки:
  1. Витягніть частоти одиночних знаків пунктуації та біграми символів (`!?`, `?!`, `...`, `!!!`).
  2. Створіть бінарні ознаки наявності “довгих хвостів” (`len(re.findall(r'!{3,}', text))`).
  3. Об’єднайте з TF-IDF (1–2-грам), навчіть `LogisticRegression(C=1.2)` та оцініть вплив на precision для сарказму.
- Підказки: використовуйте `Series.str.count`; додавайте ознаки у `scipy.sparse.csr_matrix` для ефективності.  
- Перевірка: проаналізуйте, які пунктуаційні патерни отримали найбільші коефіцієнти, і сформулюйте висновок.

---

**Варіант 5 – Детектування мемних тригерів:**  
- Мета: додати до логістичної регресії словник мемів/сленгу та перевірити їхній внесок у виявлення сарказму.  
- Кроки:
  1. Зберіть словник популярних саркастичних мемів/сленгу (наприклад, за відкритими списками).
  2. Створіть бінарні/частотні ознаки за словником, інтегруйте їх із TF-IDF у `FeatureUnion`.
  3. Навчіть логістичну регресію, порівняйте метрики й обчисліть SHAP для мемних ознак.
- Підказки: використовуйте `DictVectorizer`; слідкуйте за кодуванням (lowercase).  
- Перевірка: підготуйте топ-10 мемів із найвищими вагами та опишіть їхній вплив.

---

**Варіант 6 – Багатомовні ознаки й мовні перемикачі:**  
- Мета: обробити змішані мови та вплив мовних перемикань у саркастичних текстах.  
- Кроки:
  1. Визначте мову кожного речення (`langdetect`, `fasttext`), підрахуйте кількість мовних перемикань у межах коментаря.
  2. Додайте до TF-IDF бінарні ознаки “змішана мова”, “домінантна мова”, “інший алфавіт”.
  3. Навчіть логістичну регресію, порівняйте стабільність результатів за мовними підгрупами.
- Підказки: кешуйте результати мовної детекції; використовуйте `Pipeline`, щоб уникнути дубляжу.  
- Перевірка: наведіть криву ROC для англомовних vs змішаних коментарів, сформулюйте рекомендації.

---

**Варіант 7 – Контекстні пари коментарів:**  
- Мета: використати ознаки порівняння `comment` і `parent_comment` (відстань за n-грамами, TF-IDF косинус).  
- Кроки:
  1. Обчисліть косинусну схожість між TF-IDF `comment` та `parent_comment`, різницю довжин, наявність цитат.
  2. Додайте ці показники до моделей як числові ознаки.
  3. Навчіть логістичну регресію й оцініть, чи допомагає контекстна схожість виявляти сарказм.
- Підказки: для косинусної схожості використовуйте `sklearn.metrics.pairwise.cosine_similarity`; нормуйте значення.  
- Перевірка: подайте scatterplot “схожість vs саркастичність” і проаналізуйте.

---

**Варіант 8 – Семантичні ембедінги + логістична регресія:**  
- Мета: поєднати TF-IDF із контекстними ембедінгами (наприклад, `sentence-transformers`) для кращої класифікації.  
- Кроки:
  1. Згенеруйте sentence embeddings (наприклад, `all-MiniLM-L6-v2`) для `comment`.
  2. Об’єднайте їх із TF-IDF матрицею через `FeatureUnion` (спочатку зменшивши розмірність ембедінгів PCA до 100).
  3. Навчіть `LogisticRegression(C=0.5)` і порівняйте з чистим TF-IDF.
- Підказки: зберігайте ембедінги як `np.float32`; використовуйте `StandardScaler` перед об’єднанням.  
- Перевірка: оцініть приріст F1 і наведіть приклади, де ембедінги допомогли.

---

**Варіант 9 – Стилометричні ознаки авторів:**  
- Мета: додати стилометрію (середня довжина слова, різноманіття, типові суфікси) до логістичної регресії.  
- Кроки:
  1. Обчисліть лексичне різноманіття (`unique_tokens / total_tokens`), середню довжину слова, кількість емодзі.
  2. Додайте ці ознаки до TF-IDF у `ColumnTransformer`.
  3. Навчіть модель, оцініть вплив на precision та recall.
- Підказки: обробляйте токенізацію `nltk.word_tokenize`; для емодзі використовуйте `regex`.  
- Перевірка: продемонструйте кореляцію стилометричних ознак із сарказмом.

---

**Варіант 10 – Динамічний вибір порога за вартістю помилок:**  
- Мета: оптимізувати поріг логістичної регресії з урахуванням різних “вартісних” сценаріїв.  
- Кроки:
  1. Визначте матрицю вартостей (наприклад, FP=5, FN=1).
  2. На валідації переберіть пороги 0–1 з кроком 0.01, обчисліть очікувані витрати.
  3. Оберіть поріг із мінімальними витратами та задокументуйте результати.
- Підказки: використовуйте `numpy` для векторизованого підрахунку; побудуйте графік “поріг vs витрати”.  
- Перевірка: порівняйте метрики при новому порозі зі стандартним 0.5.

---

**Варіант 11 – Feature hashing для масштабних експериментів:**  
- Мета: прискорити обробку великого словника через `HashingVectorizer` та порівняти з класичним TF-IDF у логістичній регресії.  
- Кроки:
  1. Реалізуйте два пайплайни: TF-IDF + `LogisticRegression` і `HashingVectorizer` + `LogisticRegression`.
  2. Проведіть заміри часу та пам’яті на 5-кратній валідації (наприклад, через `memory_profiler`).
  3. Порівняйте якість і ресурси, сформулюйте висновки.
- Підказки: у `HashingVectorizer` увімкніть `alternate_sign=False`; контролюйте колізії.  
- Перевірка: оформіть таблицю “метрика/час/пам’ять” для обох підходів.

---

**Варіант 12 – Інкрементальне навчання для потокових даних:**  
- Мета: реалізувати й оцінити інкрементальне оновлення логістичної регресії на потокових батчах.  
- Кроки:
  1. Розбийте дані на послідовні батчі (наприклад, кожні 50 тис. рядків), використайте `HashingVectorizer`.
  2. По черзі застосовуйте `partial_fit` логістичної регресії (solver='saga') з підтримкою обробки потоків.
  3. Порівняйте якість і час із повним навчанням на всьому наборі.
- Підказки: збережіть стан моделі після кожного батча; використовуйте `StandardScaler` з `partial_fit`.  
- Перевірка: побудуйте графік еволюції F1 після кожного батча.

---

**Варіант 13 – Крос-перевірка з перекриттям тематичних модулів:**  
- Мета: дослідити, чи логістична регресія узагальнюється між різними тематичними групами (наприклад, політика, спорт).  
- Кроки:
  1. Розмітьте тексти тематичними тегами (можна за ключовими словами або кластеризацією TF-IDF + KMeans).
  2. Проведіть cross-topic валідацію: тренуйте на одних темах, тестуйте на інших (наприклад, leave-one-topic-out).
  3. Порівняйте результати з класичною крос-валідацією.
- Підказки: використовуйте `GroupKFold`; збережіть розподіл тем.  
- Перевірка: підготуйте висновок про теми, на яких модель узагальнюється найгірше, і запропонуйте покращення.

---

**Варіант 14 – Оцінка взаємодій через PolynomialFeatures:**  
- Мета: інтегрувати нелінійні взаємодії між числовими ознаками (наприклад, тональність × CAPS) у логістичну регресію.  
- Кроки:
  1. Витягніть числові ознаки (тональність, пунктуація, CAPS).
  2. Застосуйте `PolynomialFeatures(degree=2, interaction_only=True)` перед логістичною регресією.
  3. Порівняйте метрики з моделлю без взаємодій, звертаючи увагу на стабільність.
- Підказки: стандартизуйте ознаки перед поліноміальним розширенням; контролюйте кількість ознак.  
- Перевірка: опишіть взаємодії з найбільшими коефіцієнтами та їхню роль у сарказмі.

---

**Варіант 15 – Адаптивна відсічка словника за інформативністю:**  
- Мета: автоматично обмежити словник TF-IDF, відкидаючи малоінформативні ознаки за допомогою `SelectFromModel`.  
- Кроки:
  1. Побудуйте TF-IDF (1–3-грам) з великою кількістю ознак (до 100 тис.).
  2. Навчіть проміжну логістичну регресію та застосуйте `SelectFromModel` за порогом ваг (наприклад, top 10%).
  3. На відібраних ознаках навчіть фінальну модель, порівняйте якість та час.
- Підказки: використовувати `max_iter=1000`, щоб модель сходилася; контролюйте sparse-формат.  
- Перевірка: підготуйте графік залежності F1 від розміру словника.

---

**Варіант 16 – Змішане кодування: TF-IDF + bag-of-entities:**  
- Мета: додати до логістичної регресії інформацію про назви сутностей (імена, організації).  
- Кроки:
  1. Використайте `spaCy` або `stanza` для NER; створіть список сутностей у кожному коментарі.
  2. Побудуйте bag-of-entities (OneHot/TF-IDF по сутностях) і додайте до словних TF-IDF.
  3. Навчіть логістичну регресію й оцініть, чи допомагає знання сутностей.
- Підказки: нормуйте сутності (lowercase), об’єднуйте рідкісні в категорію `other`.  
- Перевірка: наведіть приклади, де згадки конкретних сутностей викликали саркастичні висловлювання.

---

**Варіант 17 – Аугментація даних через парафрази:**  
- Мета: перевірити, чи покращує логістичну регресію аугментація саркастичних текстів парафразами.  
- Кроки:
  1. Згенеруйте парафрази саркастичних коментарів (наприклад, `back-translation` або `parrot paraphraser`), зберігши мітку.
  2. Об’єднайте оригінальний і аугментований корпус, побудуйте TF-IDF + `LogisticRegression`.
  3. Порівняйте F1, аналізуючи вплив на recall позитивного класу.
- Підказки: контролюйте якість парафраз (видаляйте повтори, фільтруйте занадто короткі); обмежте кількість аугментованих прикладів.  
- Перевірка: наведіть статистику оновленого корпусу та порівняння метрик до/після.

---

**Варіант 18 – Фільтрація шумових користувачів:**  
- Мета: виявити авторів із великою кількістю неоднозначних коментарів та дослідити, як їхнє виключення впливає на модель.  
- Кроки:
  1. Для кожного автора порахуйте частку саркастичних коментарів, кількість суперечливих (модель помилилася).
  2. Визначте “шумових” авторів (наприклад, >70% помилок), видаліть їхні коментарі та перевчіть модель.
  3. Порівняйте метрики та поясніть, чи варто застосовувати фільтрацію.
- Підказки: авторів можна знайти через колонку `author`; зберігайте проміжні таблиці.  
- Перевірка: опишіть зміну F1 та вплив на стабільність моделі.

---

**Варіант 19 – Автоматичне формування словника сарказму:**  
- Мета: побудувати власний словник саркастичних маркерів із моделі логістичної регресії та інтегрувати його в preprocessing.  
- Кроки:
  1. Навчіть базову логістичну регресію, отримайте топ-позитивні ознаки.
  2. Збережіть їх як словник сарказму, використайте для створення додаткових бінарних ознак (“містить маркер”).
  3. Додайте нові ознакі до пайплайна та перевчіть модель, порівнюючи метрики.
- Підказки: очищайте словник від стоп-слів; додавайте нові ознаки через `FunctionTransformer`.  
- Перевірка: продемонструйте список маркерів і зростання точності/recall.

---

**Варіант 20 – Розгортання та моніторинг у вигляді звіту:**  
- Мета: створити прототип “production-ready” пайплайна логістичної регресії з моніторингом якості.  
- Кроки:
  1. Побудуйте е2е-процес: preprocessing, навчання, серіалізація (`joblib`) і функція для передбачення нових текстів.
  2. Створіть модуль моніторингу: збір метрик, логування помилок, обчислення PSI для ключових ознак.
  3. Підготуйте Markdown-звіт із описом архітектури, метрик і плану ретренінгу.
- Підказки: використовуйте `mlflow` або простий логер; забезпечте відтворюваність (`random_state`).  
- Перевірка: продемонструйте приклад передбачення на нових даних і фрагмент лог-звіту з метриками.

<a class="anchor" id="lab-4.5"></a>

## <span style="color:blue; font-size:1.2em;">4.5. Практичне застосування результатів інтелектуального аналізу даних</span>

[Повернутися до змісту](#lab-4)

<span style="color:red; font-size:2em;">Завдання 5</span>

---

**Варіант 1 – Фінальна модель зі збагаченими ознаками:**  
- Мета: зібрати всі попередньо створені ознаки (TF-IDF, пунктуація, CAPS, тональність, контекст) у єдиний пайплайн логістичної регресії та оцінити фінальну продуктивність.  
- Кроки:
  1. Об’єднайте підготовлені трансформери у `ColumnTransformer`/`FeatureUnion`, створіть повний `Pipeline`.
  2. Розбийте `train-balanced-sarcasm.csv` на train/validation/test (60/20/20) з фіксованим `random_state`.
  3. Навчіть `LogisticRegression` (solver='lbfgs', C=1), зафіксуйте метрики (F1, AUC, log-loss) на валідації, оберіть найкращі гіперпараметри та протестуйте.
- Підказки: використовуйте `Pipeline` з іменованими кроками для зручності; логуванням (`mlflow` або `wandb`) відслідковуйте експерименти.  
- Перевірка: складіть підсумковий звіт із метриками й порівнянням із базовою моделлю TF-IDF + логістична регресія.

---

**Варіант 2 – Аналіз незбалансованого набору:**  
- Мета: перевірити поведінку фінальної моделі без балансування класів, оцінити необхідність ваг/ресемплінгу.  
- Кроки:
  1. Візьміть початковий (незбалансований) `train-balanced-sarcasm.csv`, збережіть індекси саркастичних/несаркастичних.
  2. Навчіть модель у трьох сценаріях: без ваг, із `class_weight='balanced'`, із oversampling (SMOTE).
  3. Порівняйте метрики (особливо recall для сарказму) та час навчання.
- Підказки: використовуйте `imblearn.pipeline`; контролюйте витік даних (SMOTE лише на train).  
- Перевірка: побудуйте таблицю порівняння й сформулюйте рекомендацію щодо балансування.

---

**Варіант 3 – Продуктивна функція передбачення:**  
- Мета: обгорнути фінальну логістичну регресію в сервісну функцію, що повертає ймовірність сарказму та пояснення ознак.  
- Кроки:
  1. Зробіть `Pipeline` серіалізованим (через `joblib`), реалізуйте функцію `predict_with_explanation(text)` із `predict_proba`.
  2. Інтегруйте `LIME` або `SHAP` для локального пояснення уведеного тексту.
  3. Додайте перевірку: на тестових прикладах забезпечте коректність, сформуйте приклади використання.
- Підказки: для SHAP використовуйте `LinearExplainer`; обмежте кількість ознак у поясненні.  
- Перевірка: задокументуйте API-функцію, наведіть 2–3 приклади з поясненнями.

---

**Варіант 4 – Довгі vs короткі тексти:**  
- Мета: показати, як фінальна модель поводиться на різних довжинах коментарів.  
- Кроки:
  1. Розбийте дані на короткі (<50 символів), середні (50–120) та довгі (>120).
  2. Виміряйте метрики (F1, precision, recall) на кожній підмножині.
  3. Розгляньте можливість окремих моделей або зважування ознак для проблемних довжин.
- Підказки: візуалізуйте результати bar chart; при потребі навчіть окремі пайплайни.  
- Перевірка: сформулюйте рекомендації щодо обробки текстів різної довжини (наприклад, видалення шуму в коротких).

---

**Варіант 5 – Класифікація за жанрами тем:**  
- Мета: оцінити логістичну регресію в розрізі тематичних підмножин (політика, спорт, розваги).  
- Кроки:
  1. Змітьте теми ключовими словами або латентними темами (`NMF`/`LDA`).
  2. Для кожної теми обчисліть метрики моделі, порівняйте стабільність.
  3. Запропонуйте модифікації для тем, де якість падає.
- Підказки: використовуйте `TopicModel` із TF-IDF; логуванням збережіть розподіл тем.  
- Перевірка: створіть тему→метрика таблицю, додайте висновки щодо адаптації моделі.

---

**Варіант 6 – Експеримент з зовнішнім тестовим набором:**  
- Мета: перевірити здатність логістичної регресії переноситися на зовнішні дані (інший сабреддіт, інша платформа).  
- Кроки:
  1. Підготуйте зовнішній тестовий набір (за можливості), очистіть його до одного формату.
  2. Застосуйте фінальну модель без донавчання, порівняйте метрики.
  3. Проаналізуйте, які ознаки втратили актуальність, запропонуйте адаптацію (наприклад, оновити словник).
- Підказки: якщо немає зовнішнього набору, імітуйте його, відокремивши частину датасету з іншими темами.  
- Перевірка: підготуйте порівняння внутрішній vs зовнішній набір і план дій при деградації.

---

**Варіант 7 – Аналіз класифікаційних помилок з поясненнями:**  
- Мета: зібрати найтиповіші помилки (FP, FN), пояснити причини й запропонувати покращення.  
- Кроки:
  1. Витягніть топ-50 FP/FN, застосуйте SHAP/LIME для пояснення.
  2. Знайдіть повторювані патерни (наприклад, сарказм із позитивною тональністю).
  3. Запропонуйте 2–3 конкретні покращення (нові ознакі, очищення, поріг).
- Підказки: оформіть таблицю з колонками текст/проблема/рішення; дотримуйтесь анонімності.  
- Перевірка: реалізуйте одну пропозицію й виміряйте зміну метрик.

---

**Варіант 8 – Вплив feature dropout:**  
- Мета: оцінити, наскільки критичними є різні групи ознак, послідовно відключаючи їх.  
- Кроки:
  1. Визначте групи ознак (TF-IDF, тональність, пунктуація, контекст).
  2. Запустіть експерименти, кожного разу відключаючи одну групу, зафіксуйте метрики.
  3. Побудуйте “waterfall chart” впливу ознак.
- Підказки: реалізуйте параметр `include_features` у пайплайні; автоматизуйте через цикл.  
- Перевірка: підготуйте звіт із впливом кожної групи та рекомендацією, які ознакі критичні.

---

**Варіант 9 – Підбір порога під бізнес-метрику:**  
- Мета: вибрати оптимальний поріг для різних сценаріїв (мінімізація FP, максимізація recall сарказму).  
- Кроки:
  1. Зберіть precision-recall та ROC-криві для валідації.
  2. Для кожної бізнес-мети підберіть поріг (наприклад, maximize F0.5, maximize recall при precision≥0.6).
  3. Протестуйте пороги на тестовому наборі.
- Підказки: використовуйте `sklearn.metrics.precision_recall_curve`; створіть таблицю поріг→метрики.  
- Перевірка: задокументуйте вибори порогів і наведені метрики на тесті.

---

**Варіант 10 – Порівняння логістичної регресії з ансамблями:**  
- Мета: перевірити, чи перевершують фінальну логістичну модель ансамблі (наприклад, XGBoost, LightGBM) при використанні тих самих ознак.  
- Кроки:
  1. Побудуйте загальний `FeatureGenerator`, який повертає матрицю ознак.
  2. Навчіть логістичну регресію й ансамблеві моделі на тих самих ознаках.
  3. Порівняйте метрики та пояснюваність, час тренування.
- Підказки: контролюйте гіперпараметри ансамблів; аналізуйте важливість ознак.  
- Перевірка: сформулюйте висновок, коли варто залишатися на логістичній регресії, а коли — переходити до ансамблів.

---

**Варіант 11 – Моніторинг в реальному часі:**  
- Мета: розробити шаблон моніторингу продуктивності моделі в “продуктивному” середовищі.  
- Кроки:
  1. Створіть функції для обчислення метрик на ковзаючому вікні (наприклад, останні 1000 прогнозів).
  2. Поставте сповіщення при падінні метрик нижче порогу.
  3. Оцініть ефективність моніторингу через імітацію зниження якості.
- Підказки: використовуйте `psutil`/`logging`; можна моделювати потоковий сценарій.  
- Перевірка: задокументуйте “runbook” (що робити при деградації).

---

**Варіант 12 – Сценарій ретренінгу:**  
- Мета: прописати процедуру ретренінгу логістичної регресії при появі нових даних.  
- Кроки:
  1. Зберіть шаблон скрипту: завантаження нових даних, об’єднання з історичними, розділення.
  2. Перенавчіть модель, порівняйте метрики старої й нової.
  3. Передбачте rollback: як повернутися до попередньої версії при погіршенні.
- Підказки: використовуйте керування версіями (Git, DVC); автоматизуйте через `Makefile`/CI.  
- Перевірка: створіть тестовий кейс ретренінгу й опишіть результати.

---

**Варіант 13 – Документація моделі:**  
- Мета: підготувати детальну документацію (Model Card) фінальної логістичної регресії.  
- Кроки:
  1. Описати data pipeline, ознакі, гіперпараметри, метрики.
  2. Додати секції: “Обмеження”, “Етичні зауваги”, “Рекомендації користувачам”.
  3. Оформити у вигляді Markdown- або PDF-звіту, додати графіки.
- Підказки: використовуйте шаблони Model Card (Google); зберігайте статистику у вигляді таблиць.  
- Перевірка: перевірте, чи документація охоплює всі аспекти відтворюваності й етики.

---

**Варіант 14 – Порівняння граничних випадків:**  
- Мета: вручну зібрати “corner cases” (іронія без ключових слів, змішані мови) та перевірити модель.  
- Кроки:
  1. Створіть тестовий набір із 50 граничних прикладів (реальні/штучні).
  2. Запустіть модель, проаналізуйте результати й пояснення.
  3. Розробіть рекомендації щодо покращення для таких випадків.
- Підказки: використовуйте ручний контроль якості; фіксуйте приклади у CSV.  
- Перевірка: оновіть модель (за потреби) й повторно протестуйте corner cases.

---

**Варіант 15 – Персоналізація для авторів:**  
- Мета: перевірити, чи допомагає персоналізована логістична регресія (додаткові ознаки автора) для частих користувачів.  
- Кроки:
  1. Для авторів із ≥10 постів додайте бінарні ознаки “автор=Х” (one-hot або embedding).
  2. Навчіть модель і порівняйте якість із базовою.
  3. Оцініть ризики (overfitting, приватність).
- Підказки: використовуйте `OneHotEncoder(handle_unknown='ignore')`; контролюйте розмір словника авторів.  
- Перевірка: подайте результати й рекомендації, чи варто персоналізувати модель.

---

**Варіант 16 – Функція батьківського контексту:**  
- Мета: додати “функцію контрасту” між `comment` і `parent_comment` (наприклад, TF-IDF різниця) до фінальної моделі.  
- Кроки:
  1. Обчисліть різницю TF-IDF векторів або “протиріччя” (наприклад, cosine distance).
  2. Додайте як числову ознаку до пайплайна.
  3. Проаналізуйте, чи покращується розпізнавання сарказму, коли контраст великий.
- Підказки: реалізуйте це як `FunctionTransformer`; нормуйте значення.  
- Перевірка: опишіть вплив на метрики й покажіть приклади.

---

**Варіант 17 – Self-training на невизначених даних:**  
- Мета: апробувати напівконтрольоване навчання (self-training) для розширення даних.  
- Кроки:
  1. Візьміть unlabeled дані (якщо доступні) або використайте коментарі без мітки.
  2. На першому колі навчіть модель, виберіть приклади з високою впевненістю, додайте їх до тренувального набору.
  3. Повторіть 1–2 ітерації, порівняйте метрики.
- Підказки: контролюйте пороги впевненості; уникайте накопичення помилок.  
- Перевірка: оцініть приріст F1 і чи виправданий self-training.

---

**Варіант 18 – Explainable dashboard:**  
- Мета: створити інтерактивний дашборд (наприклад, `streamlit`), що демонструє можливості моделі.  
- Кроки:
  1. Побудуйте UI з полем вводу, виводом ймовірності та пояснення (SHAP/LIME).
  2. Додайте вкладку з метриками й графіками (криві ROC, confusion matrix).
  3. Забезпечте обробку помилок, логування введень.
- Підказки: використовуйте кешування в streamlit; серіалізуйте модель перед тим.  
- Перевірка: протестуйте дашборд на випадкових прикладах, переконайтеся, що пояснення коректні.

---

**Варіант 19 – Гібридний порівняльний аналіз:**  
- Мета: порівняти логістичну регресію з нейромережевою моделлю (наприклад, простим BiLSTM) при однакових даних.  
- Кроки:
  1. Підготуйте однаковий набір ознак (словникові індекси чи ембедінги).
  2. Навчіть логістичну регресію та BiLSTM, порівняйте метрики, час і пояснюваність.
  3. Опишіть компроміси між підходами.
- Підказки: використовуйте `Keras`/`PyTorch` для BiLSTM; контролюйте однаковий train/validation split.  
- Перевірка: зведіть результати у таблицю, сформулюйте висновки для курсового проєкту.

---

**Варіант 20 – Підсумкова презентація:**  
- Мета: підготувати презентаційний матеріал (слайди або ноутбук) із ключовими висновками про логістичну регресію.  
- Кроки:
  1. Зібрати ключові результати: метрики, графіки, приклади помилок, рекомендації.
  2. Оформити слайди (PowerPoint/Google Slides) або інтерактивний ноутбук (Jupyter).
  3. Переконатися, що матеріал зрозумілий для аудиторії, яка не бачила код.
- Підказки: використовуйте візуалізації (ROC, PR, SHAP); додайте розділ “Наступні кроки”.  
- Перевірка: отримайте peer-review (колега/викладач) та внесіть правки.